In [5]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

In [6]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [7]:
permutations = 128

In [8]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [9]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['Tags']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [10]:
db = pd.read_csv('tags.csv')
db['text'] = db['Stories'] + ' ' + db['Tags']
forest = get_forest(db, permutations)

It took 0.03898811340332031 seconds to build forest.


In [11]:
num_recommendations = 2
title = 'System should relate  property new liens that are purchased against previously owned liens. Should be able to see all related liens together'
result = predict(title, db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.006975650787353516 seconds to query forest.

 Top Recommendation(s) is(are) 
 18      EIN
3     Liens
Name: Tags, dtype: object


In [8]:
import pickle
with open('Tags.pickle', 'wb') as f:
    pickle.dump(forest, f)